<a href="https://colab.research.google.com/github/Draavos/making-it-rain/blob/main/vina_docking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title Comply with google driver for input and save data (optional)
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [1]:
#@title Install software
%%bash
set -e
# setup conda
  if [ ! -f CONDA_READY ]; then
    wget -qnc https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local 2>&1 1>/dev/null 
    rm Miniconda3-latest-Linux-x86_64.sh
    touch CONDA_READY
  fi
conda install -q -y -c conda-forge -c anaconda numpy boost-cpp swig openbabel rdkit scipy 2>&1 1>/dev/null
pip install meeko 2>&1 1>/dev/null
pip install PyDrive 2>&1 1>/dev/null
wget -qnc https://ccsb.scripps.edu/adfr/download/1028/ 2>&1 1>/dev/null
mv index.html ADFRsuite_Linux-x86_64_1.0_install 2>&1 1>/dev/null
chmod a+x ADFRsuite_Linux-x86_64_1.0_install 2>&1 1>/dev/null
./ADFRsuite_Linux-x86_64_1.0_install --prefix /content --mode silent 2>&1 1>/dev/null
tar -xf /content/ADFRsuite_x86_64Linux_1.0.tar.gz 2>&1 1>/dev/null
cd /content/ADFRsuite_x86_64Linux_1.0
yes y |./install.sh -d myFolder2 -c 1 2>&1 1>/dev/null 
cp "/content/ADFRsuite_x86_64Linux_1.0/myFolder2/bin/pythonsh" "/bin"
cp "/content/ADFRsuite_x86_64Linux_1.0/myFolder2/bin/prepare_receptor" "/bin"
cp "/content/ADFRsuite_x86_64Linux_1.0/myFolder2/bin/prepare_ligand" "/bin"
cp "/content/ADFRsuite_x86_64Linux_1.0/myFolder2/bin/autogrid4" "/bin"
cp "/content/ADFRsuite_x86_64Linux_1.0/myFolder2/bin/agfr" "/bin"
cd /content
git clone https://github.com/ccsb-scripps/AutoDock-Vina.git 2>&1 1>/dev/null
cp "/content/ADFRsuite_x86_64Linux_1.0/myFolder2/bin/pythonsh" "/bin"
cd /content
mkdir /content/mydata
wget -qnc  https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.3/vina_1.2.3_linux_x86_64
chmod a+x vina_1.2.3_linux_x86_64
./vina_1.2.3_linux_x86_64 --version

Cloning into 'AutoDock-Vina'...
AutoDock Vina v1.2.3


In [6]:
#@title **Please, provide the necessary input files below:** 
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
import os
PDB_filename = 'rank11.pdb' #@param {type:"string"}
ligand_filename = 'Conformer3D_CID_33042.sdf' #@param {type:"string"}
ligand2_filename = 'ahba3d.sdf' #@param {type:"string"}
ligand3_filename = '' #@param {type:"string"}

workDir = '/content/mydata' #@param {type:"string"}

pdb = os.path.join(workDir, str(PDB_filename))
ligand = os.path.join(workDir, str(ligand_filename))
ligand2 = os.path.join(workDir, str(ligand2_filename))
ligand3 = os.path.join(workDir, str(ligand3_filename))

pdb_check = os.path.exists(pdb)


if pdb_check == True:
    print("Files loaded succesfully! ;-)")
else:
    print("ERROR! Check your input names and  path")

#@markdown ---

Files loaded succesfully! ;-)


In [7]:
#@title **Prepare ligand:** 
pdb_out = os.path.join(workDir, "pdb.pdbqt")
ligand1_out = os.path.join(workDir, "ligand_in_1.pdbqt")
ligand2_out = os.path.join(workDir, "ligand_in_2.pdbqt")
ligand3_out = os.path.join(workDir, "ligand_in_3.pdbqt")
command0 = """prepare_receptor""" + " "+ "-r" + " " + str(pdb) + " " +  "-o" + " " + str(pdb_out)
command1 = """mk_prepare_ligand.py""" + " "+ "-i" + " " + str(ligand) + " " +  "-o" + " " + str(ligand1_out)
if ("sdf"or "mol2" or "mol") in ligand2:
  command2 = """mk_prepare_ligand.py""" + " "+ "-i" + " " + str(ligand2) + " " +  "-o" + " " + str(ligand2_out)
  if ("sdf"or "mol2" or "mol") in ligand3:
    command3 = """mk_prepare_ligand.py""" + " "+ "-i" + " " + str(ligand3) + " " +  "-o" + " " + str(ligand3_out)
  else:
    command3=""
    ligand3_out= ""
    print("ligand3 dont exist")

      
else :
  command3 = ""
  command2 = ""
  ligand3_out= ""
  ligand2_out= ""
  print("ligand2 and 3 dont exist")
original_stdout = sys.stdout

with open('pdbqt.sh', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(command0)
    print(command1)
    print(command2)
    print(command3)
    sys.stdout = original_stdout # Reset the standard output to its original value
!chmod 700 pdbqt.sh 2>&1 1>/dev/null
!bash pdbqt.sh


ligand3 dont exist
adding gasteiger charges to peptide


In [ ]:
#@title **Run:** 
result_out = os.path.join(workDir, "result.pdbqt")
command5 = """./vina_1.2.3_linux_x86_64""" + " "+ "--receptor" + " " + str(pdb_out) + " " +  "--ligand" + " " + str(ligand1_out) + " " + str(ligand2_out) +" " + str(ligand3_out) + " " + "--center_x 0" + " " + "--center_y 0" + " " + "--center_z 0" + " "+ "--size_x 60" + " " + "--size_y 60" + " " + "--size_z 60" +" " +"--num_modes 30"+" "+ "--exhaustiveness 64" + " " + "--out" + " " + str(result_out)
original_stdout = sys.stdout

with open('run.sh', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(command5)

    sys.stdout = original_stdout # Reset the standard output to its original value
    sys.stdout = original_stdout # Reset the standard output to its original value
!chmod 700 run.sh 2>&1 1>/dev/null
!bash run.sh


AutoDock Vina v1.2.3
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V

In [ ]:
##!pythonsh /content/AutoDock-Vina/example/autodock_scripts/prepare_gpf.py -l ahba3d.pdbqt -r btrh.pdbqt  -o btrh_ahba_test3.gpf

In [ ]:
! ./vina_1.2.3_linux_x86_64 --receptor /content/btrh1_rigid.pdbqt --flex /content/btrh1_flex.pdbqt --ligand --ligand /content/mydata/ribo.pdbqt /content/mydata/ahba.pdbqt\
--center_x 0 --center_y 0 --center_z 0 --size_x 60 --size_x 60 --size_x 60 --num_modes 30  \
--exhaustiveness 64 --out /content/mydata/ribo_ahba_rank11.pdbqt

In [32]:
from google.colab import files
files.download("/content/mydata/result.pdbqt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>